In [1]:
# Client

import socket
import base64
import hashlib
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.serialization import Encoding, PrivateFormat, NoEncryption
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.fernet import Fernet

# Create a socket
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Connect to the server
server_address = ('localhost', 12341)
client_socket.connect(server_address)

# Define hardcoded public keys for the client and server
client_public_key = 7  # Replace with a real public key
server_public_key = 5  # Replace with a real public key

# Send the client's public key to the server
client_socket.send(str(client_public_key).encode())

print("Waiting for the server to send the secret...")

# Receive the server's public key
server_public_key_received = int(client_socket.recv(1024).decode())

# Calculate the shared secret
shared_secret = (server_public_key_received * client_public_key) % 23  # Replace 23 with your prime number

if shared_secret == 12:  # Replace 12 with the expected shared secret
    print("Key exchange successful. Secrets match.")
    print("Sending an encrypted message to the server...")

    # Derive an encryption key from the shared secret using PBKDF2
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        iterations=100000,
        salt=b'',
        length=32,
    )
    encryption_key = base64.urlsafe_b64encode(kdf.derive(shared_secret.to_bytes(32, byteorder='big')))

    # Create the message
    message = "Hello hamza. thanks"
    
    # Calculate the SHA-256 hash of the message
    message_hash = hashlib.sha256(message.encode()).digest()
    print("Message Hash: ", message_hash)
    
    # Append the hash to the message
    message_with_hash = message.encode() + message_hash
    
    # Encrypt the message with the hash
    cipher = Fernet(encryption_key)
    encrypted_message = cipher.encrypt(message_with_hash)
    print("Encrypted Message: ", encrypted_message)
    client_socket.send(encrypted_message)

else:
    print("Key exchange failed. Secrets do not match.")

# Close the client socket
client_socket.close()

Waiting for the server to send the secret...
Shared Secret: 12
Key exchange successful. Secrets match.
Sending an encrypted message to the server...
Message Hash:  b"\xf4s\xae\x95!%\x1b\xdd\xcb\x96\x10p\x99\x93dtC\x04\x9c\xf6'\r1\xbdT8\xdco\r\x10\x07\xea"
Encrypted Message:  b'gAAAAABlVdECrrSzJN5_3QdGxAinbcoyR3OvcSW8hvnl4K1vUU8t93o_Q-nZl4FGTIe0zFWhteUnRspccaANYMHMtFnOiuA8r_5UGGP-G8fCGA8YYDsgtMc_NNlQuMGwukbRASNbtNhBFYNOqwpk1-3-jQxu_XTbLw=='
